# Bollinger Bands Strategy and Backtesting

Prepared by Samalie Piwan   
Email : spiwan@andrew.cmu.edu

## 1. Backtest Implementation

Load the libraries to be used in the project

In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
import seaborn as sns
import yfinance as yf
import requests
from io import StringIO
from backtesting import Backtest, Strategy
import backtesting
from tabulate import tabulate
import csv
backtesting.set_bokeh_output(notebook=False)

Declare the Simple Moving Average Window  and Standard Deviation to run the first version of the strategy with

In [121]:
# Declare the variables that will be used globally as SMA window, Short Period and Long Period / Standard Deviation
sma_window = 20
std = 3

Set the fiat currency pair to run the first version of the strategy on

In [108]:
from_symbol = "EUR"
to_symbol = "USD"
interval = "60min"
api_key = "CV4O3KUIMS9TVCLR"

### 1.1 Strategy and Returns on Annual FX data  - Jan 2005 to Dec 2013

Create a list that will hold the results from the backtest and cross-validation

In [109]:
backtest_results = []

Create a list of the currency pairs

In [110]:
currency_pairs = ["EUR/USD", "GBP/USD", "USD/JPY", "AUD/USD", "USD/CAD", "USD/CHF", "NZD/USD", "EUR/JPY", "GBP/JPY", "EUR/GBP", "EUR/CHF", "USD/SGD", "AUD/JPY", "CAD/JPY", "GBP/AUD", "NZD/JPY", "EUR/AUD", "USD/MXN", "GBP/CHF", "AUD/CHF"]

Define the Indicator(Bollinger Bands) and Buy / Sell Strategy that the backtesting.py library will optimize

In [111]:
def indicator(data, window, std_dev):
    bbands = ta.bbands(close = data.Close.s, length = window, std = std_dev)
    return bbands.to_numpy().T[:3]

class BollingerBandsStrategy(Strategy):
        
    sma_window = 20
    std = 2
    
    def init(self):
        self.bbands = self.I(indicator, self.data, self.sma_window, self.std)
        
    def next(self):
        lower_band = self.bbands[0]
        upper_band = self.bbands[2]

        if self.position:
            if self.data.Close[-1] > upper_band[-1]:
                self.position.close()
        else:
            if self.data.Close[-1] < lower_band[-1]:
                self.buy()

Define the function that will run the backtests for the currency pair list above

In [112]:
def test_strategy(start_date, end_date, param_cash, param_commission):
    for pair in currency_pairs:
        symbols = pair.split('/')
        from_symbol = symbols[0].strip()
        to_symbol = symbols[1].strip()
        # Load the monthly data from the API
        monthly_url = f"https://www.alphavantage.co/query?function=FX_MONTHLY&from_symbol={from_symbol}&to_symbol={to_symbol}&apikey={api_key}&datatype=csv"
        monthly_request = requests.get(monthly_url)
        
        #Convert the CSV to a pandas dataframe
        monthly_data = StringIO(monthly_request.text)
        csv_res_monthly_df = pd.read_csv(monthly_data)

        monthly_data_df = csv_res_monthly_df.copy()
        monthly_data_df['timestamp'] = pd.to_datetime(monthly_data_df['timestamp'])
        monthly_pricing_df = monthly_data_df.set_index('timestamp')
        monthly_pricing_df.sort_index(ascending=True, inplace=True)
        
        #Seperate the data into two sets, 2005 to 2013 and 2014 to 2022
        dataset = pd.DataFrame(monthly_pricing_df.loc[start_date : end_date].copy())
        
        dataset = dataset.rename(columns = {'open':'Open', 'high':'High', 'low':'Low', 'close':'Close'})
        
        bt = Backtest(dataset, BollingerBandsStrategy, cash = param_cash, commission = param_commission)
        stats = bt.run()
        
        # Append the PnL, Final Equity, Sharpe Ratio and Return % to the list
        
        backtest_results.append(list([pair, round(stats['_trades']['PnL'][0],3), stats['Equity Final [$]'],
                                round(stats['Sharpe Ratio'], 5), round(stats['Return [%]'], 3)]))
        
    print(backtest_results)
    return backtest_results

In [113]:
header = ["Currency Pair", "PnL", "Final Equity", "Sharpe Ratio", "Return Pct"]
strategy_results = test_strategy('2005-01-01','2013-12-31', 10000,0.01)

KeyError: 'timestamp'

In [ ]:
with open('backtest_results.csv', 'w') as results_file:
    writer = csv.writer(results_file)
    writer.writerow(header)
    writer.writerows(strategy_results)

### 2.2 Optmizing strategy to select best-performing hyperparameters

Run the backesting.py **optimize()** function using the Simple Moving Average Window and Standard Deviation as the parameters to optmize

In [114]:
optimized_stats, heatmap = bt.optimize(
    sma_window = range(20, 31, 1),
    std = list(np.round(np.linspace(1, 3, 11), 1)),
    maximize = 'Equity Final [$]',
    return_heatmap = True
)

hm = heatmap.groupby(["sma_window", "std"]).mean().unstack()

NameError: name 'bt' is not defined

Create a second table with the Sharpe Ratio, Proft and Loss (PnL) and Return percentage from the optimized backtest

In [115]:
optimized_backtest_results_table = [["Peak Equity", optimized_stats['Equity Peak [$]']],["Final Equity",optimized_stats['Equity Final [$]']],["Sharpe Ratio", round(optimized_stats['Sharpe Ratio'], 5)], 
                          ["PnL", f"${round(optimized_stats['_trades']['PnL'][0],3)}"],
                          ["Return %", f"{round(optimized_stats['Return [%]'], 3)}%"]]
print(f"Backesting reults with SMA Window and Standard Deviation optimized\n")
print(tabulate(optimized_backtest_results_table, headers=["Result", "Value"]))                         

NameError: name 'optimized_stats' is not defined

Plot a heatmap of the best-performing hyperparameters

In [104]:
pricing_hm = sns.heatmap(hm[::-1], cmap='viridis')
pricing_hm.set(xlabel='Standard Deviation', ylabel='SMA Window')
plt.show()

#plot_heatmaps(heatmap, agg='mean')

NameError: name 'hm' is not defined

### 2.3 Cross validation

Get the optimal parameters

In [116]:
optmized_std = optimized_stats['_strategy'].std
optimized_window = optimized_stats['_strategy'].sma_window

Run an instance of our Backtest class using  from section 2 using:
- The **'rerun_dataset'** from section 2
- Cash of $10,000
- Commission at 1%

In [117]:
class OptBollingerBandsStrategy(Strategy):
    
    std_cust = optmized_std
    sma_window_cust = optimized_window
    
    def init(self):
        self.bbands = self.I(indicator, self.data, self.std_cust, self.sma_window_cust)

    def next(self):
        lower_band = self.bbands[0]
        upper_band = self.bbands[2]
        
        if self.position:
            if self.data.Close[-1] > upper_band[-1]:
                self.position.close()
                pnl = main_df_indexed['open'][i] - main_df_indexed['close'][i]
                pnl_list.append(pnl)
        else:
            if self.data.Close[-1] > lower_band[-1]:
                self.position.close()
                self.buy()
                pnl = main_df_indexed['close'][i] - main_df_indexed['open'][i]
                pnl_list.append(pnl)

In [118]:
bt_opt = Backtest(rerun_dataset, OptBollingerBandsStrategy, cash=10000, commission=0.01)
stats_opt = bt_opt.run()

NameError: name 'rerun_dataset' is not defined

Create a table with the Sharpe Ratio, Proft and Loss (PnL) and Return percentage from the backtest

In [104]:
optimized_backtest_results_table = [["Peak Equity", stats_opt['Equity Peak [$]']],["Final Equity",stats_opt['Equity Final [$]']],["Sharpe Ratio", round(stats_opt['Sharpe Ratio'], 5)], 
                          ["PnL", f"${round(stats_opt['_trades']['PnL'][0],3)}"],
                          ["Return %", f"{round(stats_opt['Return [%]'], 3)}%"]]
print(f"Backesting optimized reults with investment of {10000} and commission of 1%\n")
print(tabulate(optimized_backtest_results_table, headers=["Result", "Value"]))                         

Backesting optimized reults with investment of 10000 and commission of 1%

Result        Value
------------  ------------------
Peak Equity   11000.888840699998
Final Equity  9221.1984407
Sharpe Ratio  0.0
PnL           $-778.802
Return %      -7.788%
